Лабораторная работа. MySQL.

Эта лабораторная работа имеет целью эмулировать процесс обработки данных, загрузку их в базу данных, обновление этих данных в базе, составление запросов к ней и выгрузку данных. Создавая и заполняя таблицы, проверяйте появление тех или иных элементов или записей в таблице в MySQL Workbench для большей наглядности.

Смодулируем небольшое исследование, которое разметит тексты по нескольким метрикам. Для этого используем небольшой датасет критики детской литературы отсюда: https://dataverse.pushdom.ru/dataset.xhtml?persistentId=doi:10.31860/openlit-2022.12-B008

**Глава 1. Метаданные**

Пушкинский дом подготавливает для нас описание корпуса, поэтому можно использовать табличку с метаданными для нашей работы.

In [1]:
import pandas as pd

In [8]:
path = 'critics/bibliography.tsv' #измените путь на свой
meta = pd.read_csv(path, sep='\t') #пушкинский дом выкладывает файлы в  tsv (tab separated values), поэтому надо указать правильный делиметер
meta

,№,author,title,source,year,bibliography
0,1,А. П. Г.,О русской детской литературе,Возрождение,1932,А. П. Г. О русской детской литературе // Возро...
1,2,Бобринская В. Н.,Библиотека для детей: письмо в редакцию,Возрождение,1933,Бобринская В. Н. Библиотека для детей: письмо ...
2,3,Бобринская В. Н.,Наш долг перед русскими детьми,Возрождение,1926,Бобринская В. Н. Наш долг перед русскими детьм...
3,4,Бобринская В. Н.,Русский педагогический музей в Париже: отчет з...,Возрождение,1930,Бобринская В. Н. Русский педагогический музей ...
4,5,Вейдле В. В.,Сказки для детей,Возрождение,1931,Вейдле В. В. Сказки для детей // Возрождение. ...
...,...,...,...,...,...,...
245,246,Трубников П.,Письма детишек в СССР,Сегодня,1934,Трубников П. [Пильский П. М.] Письма детишек в...
246,247,Н. Н.,Ильина-Полторацкая Е. Как мы жили в старинной ...,Сегодня,1923,"Н. Н. [Рецензия] // Сегодня. 1923. № 27, 4 фев..."
247,248,Мандельштам Ю. В.,Зайцевой С. А. Детскими глазами на мир. Харбин...,Современные записки,1938,Мандельштам Ю. В. [Рецензия] // Современные за...
248,249,Алексеев Г.,"Цветень: сборник для детей, книга первая. Гран...",Сполохи,1922,Алексеев Г. [Рецензия] // Сполохи. 1922. № 5 (...


В таблице есть записи годов "1927-1928". Это не годится для типа данных INT или YEAR в SQL таблице, поэтому изменим слегка эту запись:

In [9]:
meta['year'] = meta['year'].replace(['1927-1928'], '1928')

Как видно, в табличке есть вся нужная метаинформация по корпусу. Создадим отдельную базу данных и сохраним туда эту табличку. Назначим номер первичным ключом.

In [4]:
import mysql.connector
from getpass import getpass

In [ ]:
mydb = mysql.connector.connect(
    host = 'localhost',
    user = "root",
    password = getpass('Введите ваш пароль!')  
)

mycursor = mydb.cursor()
mycursor.execute("SHOW DATABASES")

for database in mycursor:
  print(database)

('information_schema',)
('linguistics',)
('mysql',)
('newschema',)
('performance_schema',)
('royallib',)
('sakila',)
('sys',)
('world',)
('writers_3',)


In [12]:
mydb = mysql.connector.connect(
  host = "localhost",
  user = "root",
  password = getpass('Введите ваш пароль!')   
)

mycursor = mydb.cursor()
mycursor.execute("CREATE DATABASE critics") #создаём пока только саму БД с названием critics

Загрузим табличку с метаинформацией в БД. Заполним её таким же образом, как было в семинарском занятии по Python + SQL. Библиографию брать не будем. Для этого извлечем кортежи, т.е. строки, из пандасовского датафрейма.

In [13]:
for_db = [(int(meta['№'][ind]), meta['author'][ind], meta['title'][ind], meta['source'][ind], int(meta['year'][ind])) for ind in meta.index]
for_db #list comprehension для сбора кортежей для нашей таблицы

[(1, 'А. П. Г.', 'О русской детской литературе', 'Возрождение', 1932),
 (2,
  'Бобринская В. Н.',
  'Библиотека для детей: письмо в редакцию',
  'Возрождение',
  1933),
 (3,
  'Бобринская В. Н.',
  'Наш долг перед русскими детьми',
  'Возрождение',
  1926),
 (4,
  'Бобринская В. Н.',
  'Русский педагогический музей в Париже: отчет за 1929 год',
  'Возрождение',
  1930),
 (5, 'Вейдле В. В.', 'Сказки для детей', 'Возрождение', 1931),
 (6, 'И. Г. К.', '«Огоньки»', 'Возрождение', 1932),
 (7, 'И. Г. К.', 'Книга для детей', 'Возрождение', 1932),
 (8, 'И. Л.', '«Чудесное лето»', 'Возрождение', 1930),
 (9,
  'Л.',
  'Чуковский К. И. Маленькие дети. Петроград: Изд. «Кр. Газеты», 1928.',
  'Возрождение',
  1928),
 (10,
  'Ладыженский В. Н.',
  'Саша Черный. Дневник фокса Микки / рисунки Ф. Рожанковского. Изд. автора: Париж, 1927.',
  'Возрождение',
  1927),
 (11, 'С.', 'Кошачья санатория', 'Возрождение', 1928),
 (12, 'Тэффи', 'Игрушки и книги', 'Возрождение', 1927),
 (13, 'Ю. М.', 'Андерсен', 'В

Создадим пустую табличку.

Для начала посмотрим, какие таблички уже есть в БД:

In [14]:
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = getpass('Введите ваш пароль!'),
    database = 'critics'
)

mycursor = mydb.cursor()
mycursor.execute("SHOW TABLES")

for table in mycursor:
  print(table)

Пусто! Создадим таблицу.

In [15]:
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = getpass('Введите ваш пароль!'),
    database = 'critics'
)

mycursor = mydb.cursor()
mycursor.execute("CREATE TABLE meta (id INT AUTO_INCREMENT PRIMARY KEY,  author VARCHAR(1000), title VARCHAR(1000), source VARCHAR(1000), year YEAR)")

Проверим еще раз.

In [16]:
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = getpass('Введите ваш пароль!'),
    database = 'critics'
)

mycursor = mydb.cursor()
mycursor.execute("SHOW TABLES")

for table in mycursor:
  print(table)

('meta',)


Теперь заполним эту таблицу полученной ранее информацией.

In [17]:
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = getpass('Введите ваш пароль!'),
    database = 'critics'
)

mycursor = mydb.cursor()
sql = "INSERT INTO meta (id, author, title, source, year) VALUES (%s, %s, %s, %s, %s)"

mycursor.executemany(sql, for_db) 
mydb.commit() #не забудьте делать коммит, иначе не сохранится запись в БД

Посмотрим, что получилось. Пока только в Workbench.

**Глава 2. Хранение текстов в БД**

Создадим ещё одну таблицу в нашей БД, в неё мы запишем тексты критических разборов. Подготовим некоторые данные для этого.

In [3]:
import os

Создадим список из кортежей, состоящих из индекса и текста рецензии.

In [22]:
path = 'critics/critic_corpus/' #не забудьте поменять на путь, где лежат ваши тексты
texts = [] #сюда мы сохраним кортежи текстов и их индексов
counter = 1 #счётчик, который поможет нам сохранять индексы
for filename in os.listdir(path): #проходим циклом по названиям файлов в папке
    with open(path + filename, encoding='utf-8') as txt:
        texts.append((counter, txt.read()))
        counter += 1
        
texts

[(1,
  'А. П. Г. О русской детской литературе // Возрождение. 1932. 21 июля. С. 2.\nГ-н И. Г. К. в своей заметке «Книги для детей»  затронул вопрос, интересующий очень многих, вернее сказать, всех, кому приходится иметь дело с детьми. Всякому понятно, как важно дать ребенку хорошую книгу, но найти такую книгу часто бывает довольно трудно.\nНельзя не согласиться с мнением ав¬тора вышеупомянутой заметки, что всего труднее выбор книг для так называемого среднего возраста, причем средним возрастом он, повидимому, называет детей с момента, когда ребенок начинает читать сам и до 14 лет.  После 14 лет можно значительно расширить круг пригодных книг, а дли малюток, как правильно замечает г-н И. Г. К., всего нужнее книга с хорошими картинками совсем без текста.\nЗаключительные слова заметки можно оспаривать. Г-н И. Г. К. говорит, что только «новейшее» время отнеслось сознательно к детской литературе и что хороших книг для детей еще очень мало.\nДа, их мало здесь, в эмиграции, потому что все наш

Создадим табличку texts и заполним её текстами.

In [23]:
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = getpass('Введите ваш пароль!'),
    database = 'critics'
)

mycursor = mydb.cursor()
mycursor.execute("CREATE TABLE texts (id INT AUTO_INCREMENT PRIMARY KEY,  text TEXT)")
sql = "INSERT INTO texts (id, text) VALUES (%s, %s)"
mycursor.executemany(sql, texts) 
mydb.commit()

Не забудем связать таблицы через внешний ключ:

In [24]:
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = getpass('Введите ваш пароль!'),
    database = 'critics'
)

mycursor = mydb.cursor()
mycursor.execute("ALTER TABLE texts ADD FOREIGN KEY (id) REFERENCES meta(id)")

Теперь мы можем забыть о том, что у нас есть какая-то папочка и работать только с сервером. Перейдем к следующему этапу.

**Глава 3. Выгрузка данных из БД и их обработка**

Сделаем небольшие подсчёты по текстам нашей БД. Выгрузим тексты из неё.

In [25]:
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = getpass('Введите ваш пароль!'),
    database = 'critics'
)

mycursor = mydb.cursor()
sql = "SELECT text FROM texts"
mycursor.execute(sql)
myresult = mycursor.fetchall()

Тексты в myresult лежат в кортежах, достанем их из них:

In [26]:
texts_cr = []
for tupl in myresult:
   # print(tupl)
    for txt in tupl: 
        #print(txt)
        texts_cr.append(txt)
texts_cr

['А. П. Г. О русской детской литературе // Возрождение. 1932. 21 июля. С. 2.\nГ-н И. Г. К. в своей заметке «Книги для детей»  затронул вопрос, интересующий очень многих, вернее сказать, всех, кому приходится иметь дело с детьми. Всякому понятно, как важно дать ребенку хорошую книгу, но найти такую книгу часто бывает довольно трудно.\nНельзя не согласиться с мнением ав¬тора вышеупомянутой заметки, что всего труднее выбор книг для так называемого среднего возраста, причем средним возрастом он, повидимому, называет детей с момента, когда ребенок начинает читать сам и до 14 лет.  После 14 лет можно значительно расширить круг пригодных книг, а дли малюток, как правильно замечает г-н И. Г. К., всего нужнее книга с хорошими картинками совсем без текста.\nЗаключительные слова заметки можно оспаривать. Г-н И. Г. К. говорит, что только «новейшее» время отнеслось сознательно к детской литературе и что хороших книг для детей еще очень мало.\nДа, их мало здесь, в эмиграции, потому что все наши бога

Подсчитаем пару простых метрик. Например, ридабилити. Не будем вдаваться в сложности и тонкости правильных формул, воспользуемся готовыми решениями для скорости. Посчитаем метрику MTLD (одна из метрик лексического разнообразия) TTR (type-token ratio).

In [4]:
from lexical_diversity import lex_div as ld

In [30]:
lst_mtlds_ttrs = []
counter = 1
for text in texts_cr:
    tokenized_text = ld.tokenize(text)
    tokenized_text_mtld = round(ld.mtld(tokenized_text), 3)
    tokenized_text_ttr = round(ld.mattr(tokenized_text), 3)
    lst_mtlds_ttrs.append((counter, tokenized_text_mtld, tokenized_text_ttr))
    counter += 1
lst_mtlds_ttrs #эти значения мы запишем затем в нашу БД

[(1, 351.478, 0.913),
 (2, 299.908, 0.877),
 (3, 189.242, 0.86),
 (4, 137.593, 0.824),
 (5, 240.484, 0.872),
 (6, 381.251, 0.909),
 (7, 475.565, 0.918),
 (8, 207.739, 0.874),
 (9, 410.027, 0.956),
 (10, 268.561, 0.884),
 (11, 171.258, 0.918),
 (12, 327.432, 0.889),
 (13, 377.477, 0.902),
 (14, 229.955, 0.89),
 (15, 258.252, 0.881),
 (16, 546.985, 0.908),
 (17, 238.023, 0.948),
 (18, 345.857, 0.866),
 (19, 272.653, 0.944),
 (20, 406.452, 0.92),
 (21, 476.565, 0.902),
 (22, 113.4, 0.889),
 (23, 211.97, 0.865),
 (24, 389.031, 0.887),
 (25, 590.333, 0.911),
 (26, 261.032, 0.891),
 (27, 360.141, 0.854),
 (28, 198.81, 0.882),
 (29, 191.714, 0.852),
 (30, 190.997, 0.868),
 (31, 252.0, 0.864),
 (32, 243.056, 0.918),
 (33, 264.915, 0.905),
 (34, 172.093, 0.864),
 (35, 374.197, 0.905),
 (36, 254.545, 0.9),
 (37, 162.928, 0.85),
 (38, 289.0, 0.912),
 (39, 175.0, 0.909),
 (40, 324.552, 0.909),
 (41, 551.191, 0.91),
 (42, 410.423, 0.91),
 (43, 383.593, 0.933),
 (44, 360.961, 0.918),
 (45, 392.13, 0

Создадим ещё одну табличку, где будем хранить наши данные для аналитики.

In [31]:
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = getpass('Введите ваш пароль!'),
    database = 'critics'
)

mycursor = mydb.cursor()
mycursor.execute("CREATE TABLE analytics (id INT AUTO_INCREMENT PRIMARY KEY,  mtld FLOAT(3), ttr FLOAT(3))")
#mycursor.execute("ALTER TABLE analytics ADD FOREIGN KEY (id) REFERENCES meta(id)")
sql = "INSERT INTO analytics (id, mtld, ttr) VALUES (%s, %s, %s)"
mycursor.executemany(sql, lst_mtlds_ttrs) 
mydb.commit()

In [32]:
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = getpass('Введите ваш пароль!'),
    database = 'critics'
)

mycursor = mydb.cursor()
mycursor.execute("ALTER TABLE analytics ADD FOREIGN KEY (id) REFERENCES meta(id)")

**Глава 4. Аналитика.**

Все наши дальнейшие действия крайне тривиальны: нам нужно получать выгрузку и что-то анализировать. Давайте посчитаем корреляции по записям в нашей БД.

In [33]:
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = getpass('Введите ваш пароль!'),
    database = 'critics'
)

mycursor = mydb.cursor()
sql = "SELECT mtld, ttr FROM analytics"
mycursor.execute(sql)
myresult = mycursor.fetchall()

In [5]:
import numpy as np

In [35]:
mtlds = [] #сохраним отдельно mtld и ttr
ttrs = []
for tupl in myresult:
    mtlds.append(tupl[0])
    ttrs.append(tupl[1])

In [36]:
cor_matrix = np.corrcoef(mtlds, ttrs) #делаем матрицу корреляций
cor_matrix #получаем среднюю (среднеслабую) корреляцию в 0.46

array([[1.        , 0.45984862],
       [0.45984862, 1.        ]])

Так как мы в SQL, мы можем писать более хитрые запросы. Например, можно вывести значения mtld и ttr текстов, написанных в и после 1927 года. Это так называемые вложенные запросы, или подзапросы.

In [37]:
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = getpass('Введите ваш пароль!'),
    database = 'critics'
)

mycursor = mydb.cursor()
sql = "SELECT mtld, ttr FROM analytics WHERE id IN (SELECT id FROM meta WHERE year >= 1927)"
mycursor.execute(sql)
myresult = mycursor.fetchall()

In [38]:
print(len(myresult))

126


Как видно после исполнения предыдущей команды, в результат выдачи попало только 126 записей. Таким образом мы отсекли около половины текстов. Давайте снова посчитаем корреляцию:

In [39]:
mtlds = [] #сохраним отдельно mtld и ttr
ttrs = []
for tupl in myresult:
    mtlds.append(tupl[0])
    ttrs.append(tupl[1])
r = np.corrcoef(mtlds, ttrs) #делаем матрицу корреляций
r #получаем среднюю (среднеслабую) корреляцию в 0.46

array([[1.        , 0.42629508],
       [0.42629508, 1.        ]])

Довольно забавно, но степень корреляции упала. У этого нет какого-то научного или серьёзного значения, т.к. это совсем небольшое и игрушечное исследование, но тем не менее примерно так вы и будете работать в тех научных проектах, где есть БД, а также примерно такие умения начального и среднего уровнвя от вас обычно ожидается на стартовых позициях, где требуется знание SQL.

--------------------

**Глава 5. Свободное искусство (то есть задание для самостоятельной работы).**

Ваша задача выбрать какой-либо датасет (не слишком большой - вы не должны сделать итоговый проект в рамках этой работы, хотя итоговый проект во многом должен быть похож именно на эту лабораторную работу) из множества датасетов Пушкинского дома ( https://dataverse.pushdom.ru/) и обработать его.

Задания:
1) выбрать датасет;  **(Done)**
2) создать БД с несколькими таблицами, которые будут отображать всё великолепие и многообразие датасета (мета-, тексты, статистические подсчёты); **(Done)**
3) заполнить эти таблицы; **(Done)**
4) придумать небольшое исследование: например, корреляцию количества определенных частей речи с количеством повторений слова "запонка" в тексте. Достаточно 1-2 метрик, это игрушечное исследование, а не заготовка вашей статьи **(Done)**;
5) проиллюстрировать ваше умение создавать таблицы **(Done)**, заполнять таблицы **(Done)**, получать выгрузку **(Done)**, обновлять таблицы **(Done)** и данные в них **(Done)**;
6) также напишите 5-6 комбинаторных запросов. Например, "подсчёт числа текстов, написанных до 1927 года". Попробуйте придумать максимально сложные для рассматриваемого вами датасета **(Done)**;
7) Напишите хотя бы один вложенный запрос **(Done)**;
8) Сделайте один разумный JOIN и обоснуйте его **(Done)**.

Оцениваться будет ваше умение аккуратно писать и обосновывать запросы. Комментируйте большинство строчек кода, надписав, что делает эта строчка. Каждый недублирующийся SQL запрос распишите, что он последовательно делает.

Помимо семинарских занятий 3 модуля вам могут быть полезны выложенные учебники, а также онлайн-справочники, а особенно следующие  главы:
1) https://www.w3schools.com/sql/default.asp
2) https://www.w3schools.com/sql/sql_join.asp
3) https://www.w3schools.com/sql/sql_alter.asp
4) https://www.w3schools.com/sql/sql_insert.asp
5) https://www.w3schools.com/sql/sql_update.asp


Пришлите до 15.05 включительно на почту a.klimov@hse.ru архив с такой же тетрадкой, где выполнены и прокомментированы задания, а также дамп и схема вашей БД. По желанию можно прислать csv таблицы выгрузок.

In [5]:
path = 'skazki/metadata.tab' 
meta = pd.read_csv(path, sep='\t') 
meta = meta.drop(["comments", "link_uid", "cycle", "link"], axis=1) # Дропаю неинтересные мне данные, где много нулей, делаю датасет чуть меньше

# Создадим функцию для преобразования текста версии в числовой формат
def parse_version(version_text):
    if pd.isna(version_text):  # Проверка на NaN
        return int(0)
    elif "Ранняя" in str(version_text):
        return int(1)
    elif "Поздняя" in str(version_text):
        return int(2)
    else:
        return int(0)

# Применяем функцию к столбцу с версиями, создаем айди совмещающий uid и version
meta['version_num'] = meta['version'].apply(parse_version)
meta["version"] = meta['version_num']
meta["uid"] = meta["uid"] + "_" + meta["version"].astype(str)
meta = meta.drop("version_num", axis=1)  
meta = meta.drop("version", axis=1)  

# Меняем строковые данные на boolean
meta["lifetime_publication"] = meta["lifetime_publication"] == 'да'

# В датасете нашел дубликаты одинаковых стихов, которые отсылают к одним и тем же файлам, не имеют различных версия и абсолютно одинаковые, поэтому я их дропаю
meta = meta.drop_duplicates(subset="uid", keep="first")  
meta.head()

,volume,uid,title,incipit,terminus_post_quem,terminus_ante_quem,dated,autographs,lifetime_publication,first_publication,bibliography,citation,filename,n_lines,n_words
0,1,A1_0,«А в ненастные дни…»,«А в ненастные дни…»,1828,1828,1828 г. на основании письма Пушкина к Вяземско...,ПД 1363 (в письме к П. А. Вяземскому от 1 сент...,True,"БдЧ. 1834. Т. 2, кн. 3 (в качестве эпиграфа к ...",Лернер Н. О. Заметки о Пушкине // ПиС. Вып. 16...,"Акад. Т. 3, кн. 1. С. 115.",A1.txt,12.0,27.0
1,1,A2_0,Адели,"«Играй, Адель…»",1821,1821,"1821 г., предположительно около 12 апреля, по ...","ПД 833, л. 8 об. (беловой, с поправками).",True,"ПЗ 1824 (с редакторским, по-видимому, заглавие...","Иезуитова Р. В. Рабочая тетрадь Пушкина ПД, № ...","АПСС. Т. 2, кн. 2. С. 20.",A2.txt,17.0,37.0
2,1,A3_0,Акафист Екатерине Николаевне Карамзиной,«Земли достигнув наконец…»,1827,1827,31 июля — 24 ноября 1827 г. на основании помет...,"На листе, вырезанном из тетр. ПД 836 (л. 19), ...",False,Якушкин. № 6 (не полностью); более полно: ПиС....,Иезуитова Р. В. К истории декабристских замысл...,"Акад. Т. 3, кн. 1. С. 64.",A3.txt,10.0,38.0
3,1,A4_0,Аквилон,"«Зачем ты, грозный аквилон…»",1824,1830,"1824 г., после 9 августа (времени приезда Пушк...","ПД 126 (беловой с поправками, переходящий в че...",True,"ЛПРИ. 1837. № 1, 2 января.","Благой, II. С. 481–482, 694–696; Виноградов. С...","АПСС. Т. 3, кн. 1. С. 33.",A4.txt,16.0,67.0
4,1,A5_0,Алексееву,"«Мой милый, как несправедливы…»",1821,1821,Концом ноября — декабрем 1821 г. на основании ...,"ПД 833, л. 13–14 (беловой с правкой).",True,ПЗ 1825.,Бартенев. П. в южной России. С. 95; Бродский Н...,"АПСС. Т. 2, кн. 2. С. 54–55.",A5.txt,44.0,177.0


In [ ]:
# Подключаюсь
mydb = mysql.connector.connect(
    host = 'localhost',
    user = "root",
    password = getpass('Введите ваш пароль!')  
)

mycursor = mydb.cursor()
mycursor.execute("SHOW DATABASES")

for database in mycursor:
  print(database)

('critics',)
('gulag',)
('information_schema',)
('linguistics',)
('mysql',)
('newschema',)
('performance_schema',)
('royallib',)
('sakila',)
('sys',)
('world',)
('writers_3',)


In [ ]:
# Создаём БД под названием skazki
mycursor = mydb.cursor()
mycursor.execute("CREATE DATABASE skazki") 

In [ ]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password=getpass('Введите ваш пароль!'),
  database="skazki"
)

mycursor = mydb.cursor()

# Создаем таблицу с метаданными, задаем первичный ключ, типы данных для каждой хар-ки
mycursor.execute("""CREATE TABLE poems_metadata (
    PRIMARY KEY(uid), # первичный ключ по айди
    uid VARCHAR(10) ,
    title VARCHAR(255),
    incipit TEXT,
    year_start INT,
    year_end INT,
    dated TEXT,
    autographs TEXT,
    lifetime_publication BOOLEAN,
    first_publication TEXT,
    bibliography TEXT,
    citation TEXT,
    n_lines INT,
    n_words INT
)
""")


In [ ]:
# Создаем таблицу с текстами, задаем первичный и внешний ключ
mycursor.execute("""CREATE TABLE poems_texts (
    uid VARCHAR(10),
    text_content LONGTEXT,
    PRIMARY KEY (uid),  # первичный ключ по айди
    FOREIGN KEY (uid) REFERENCES poems_metadata(uid) ON DELETE CASCADE # Внешний ключ по айди в таблице метаданных
)
""")

In [ ]:
# Подготовка данных для вставки в таблицу метаданных, меняю тип данных
for_db = [
    (
        str(meta['uid'][ind]),  # uid
        str(meta['title'][ind]),  # title
        str(meta['incipit'][ind]),  # incipit
        int(meta['terminus_post_quem'][ind]) if pd.notna(meta['terminus_post_quem'][ind]) else None,  # year_start
        int(meta['terminus_ante_quem'][ind]) if pd.notna(meta['terminus_ante_quem'][ind]) else None,  # year_end
        str(meta['dated'][ind]) if pd.notna(meta['dated'][ind]) else None,  # dated
        str(meta['autographs'][ind]) if pd.notna(meta['autographs'][ind]) else None,  # autographs
        bool(meta['lifetime_publication'][ind]),  # lifetime_publication
        str(meta['first_publication'][ind]) if pd.notna(meta['first_publication'][ind]) else None,  # first_publication
        str(meta['bibliography'][ind]) if pd.notna(meta['bibliography'][ind]) else None,  # bibliography
        str(meta['citation'][ind]) if pd.notna(meta['citation'][ind]) else None,  # citation
        int(meta['n_lines'][ind]) if pd.notna(meta['n_lines'][ind]) else None,  # n_lines
        int(meta['n_words'][ind]) if pd.notna(meta['n_words'][ind]) else None  # n_words
    )
    for ind in meta.index
]

# Вставка данных в таблицу метаданных, добавляю данные в столбцы uid, title, incipit, year_start, year_end, dated, autographs, lifetime_publication, first_publication, bibliography, citation, n_lines, n_words
sql = """
INSERT INTO poems_metadata 
(uid, title, incipit, year_start, year_end, dated, autographs, 
 lifetime_publication, first_publication, bibliography, citation, n_lines, n_words) 
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

mycursor.executemany(sql, for_db)
mydb.commit()

In [ ]:
path = 'skazki/texts/' 
texts = [] 
# Полготавливаю пары uid - text для вставки в таблицу poems_texts
for filename in os.listdir(path): #проходим циклом по названиям файлов в папке
    for index, row in meta.iterrows():
        if filename == row['filename']: # Сопоставляем uid и text
            with open(path + filename, encoding='utf-8') as txt:
                texts.append((row['uid'], txt.read()))
        
texts

[('A1_0',
  'А в ненастные дни\nСобирались они\nЧасто.\nГнули, <- - - -> их <- - ->!\nОт пятидесяти\nНа сто.\nИ выигрывали\nИ отписывали\nМелом.\nТак в ненастные дни\nЗанимались они\nДелом.'),
 ('A10_0',
  'Увы! напрасно деве гордой\nЯ предлагал свою любовь!..\nНи наша жизнь, ни наша кровь\nЕе души не тронет твердой.\nСлезами только буду сыт,\nХоть сердце мне печаль расколет.\nОна на щепочку насцит,\n<Но> и понюхать не позволит.'),
 ('A100_0',
  'Все призрак, суета\nВсе дрянь и гадость;\nСтакан и красота —\nВот жизни радость.\n\nЛюбовь и вино\nНам нужны равно;\nБез них человек\nЗевал бы весь век.\n\nК ним лень еще прибавлю,\nЛень с ими заодно;\nВино я с нею славлю,\nОна мне льет вино.'),
 ('A101_0',
  'Все так же осеня<ют> своды\n<             > трех цариц,\nВсе те же клики юных жриц,\nВсе те же вьются хоров<оды>.\nУжель умолк волшебный глас\nСеменовой, сей чудной музы?\nУжель, навек оставя нас,\nОна расторгла с Фебом узы\nИславы русской луч угас?\nНе верю! вновь о<на><?> <нрзб> —\nЕй 

In [ ]:
# Вставляем в таблицу текстов тексты и айди
sql = "INSERT INTO poems_texts (uid, text_content) VALUES (%s, %s)"
mycursor.executemany(sql, texts) 
mydb.commit()

In [ ]:
# Создаем табличку, в которую вставим данные по частям речи, задаем первичный ключ по uid, внешний ключ также по uid. Добавляю каскадное удаление.
mycursor.execute("""CREATE TABLE poems_pos (
    uid VARCHAR(10),
    noun_count INT,
    verb_count INT,
    adj_count INT, 
    most_popular_pos VARCHAR(5),
    PRIMARY KEY (uid),  
    FOREIGN KEY (uid) REFERENCES poems_metadata(uid) ON DELETE CASCADE
)
""")

In [ ]:
# Понял, что нужно поменять количество знаков в most_popular_pos, не хватает для метки "adjectives"
mycursor.execute("""ALTER TABLE poems_pos 
MODIFY COLUMN most_popular_pos VARCHAR(10)""")

In [15]:
import re
from pymorphy3 import MorphAnalyzer
import spacy

In [16]:
# Инициализация инструментов анализа
nlp = spacy.load("ru_core_news_lg")
morph = MorphAnalyzer()


In [ ]:
# Получаем тексты для анализа
mycursor.execute("SELECT uid, text_content FROM poems_texts")
texts = mycursor.fetchall()

In [ ]:
# Функция для анализа количества частей речи

def count_pos(text):

    # Очищаем текст
    cleaned_text = re.sub(r'[^а-яё\s]', '', text.lower())
    
    # Лемматизируем слова с помощью Pymorphy3
    lemmas = []
    for word in cleaned_text.split():
        parsed_word = morph.parse(word)[0]  
        lemmas.append(parsed_word.normal_form)
    
    # Анализируем части речи с помощью Spacy
    doc = nlp(" ".join(lemmas))
    
    # Считаем части речи
    pos_counts = {
        'noun': 0,
        'adjective': 0,
        'verb': 0,
        'most_popular_pos': 0
    }
    
    for token in doc:
        if token.pos_ == 'NOUN':
            pos_counts['noun'] += 1
        elif token.pos_ == 'ADJ':
            pos_counts['adjective'] += 1
        elif token.pos_ == 'VERB':
            pos_counts['verb'] += 1

    # Выводим самую частотную часть речи
    max_pos = max(pos_counts, key=pos_counts.get)
    pos_counts['most_popular_pos'] = max_pos
    
    return pos_counts

In [ ]:
# Микро- проверка

count_pos(texts[1][1])

{'noun': 10, 'adjective': 2, 'verb': 6, 'most_popular_pos': 'noun'}

In [ ]:
# Функция для отправки данных по частям речи в БД

def insert_pos_into_db(texts):
    # Подключаемся к БД
    mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = getpass('Введите ваш пароль!'),
    database = 'skazki')    
    mycursor = mydb.cursor()

    for uid, text in texts:

        # Получаем данные из текста
        pos_data = count_pos(text)

        # Вставляем данные 
        sql = """INSERT INTO poems_pos 
                      (uid, noun_count, verb_count, adj_count, most_popular_pos) 
                      VALUES (%s, %s, %s, %s, %s)"""
            
        # Сами данные
        text_pos = (
                uid,
                pos_data['noun'],
                pos_data['verb'],
                pos_data['adjective'],
                pos_data['most_popular_pos']
            )   
        
        # Выполняем запрос
        mycursor.execute(sql, text_pos)
        mydb.commit()

In [ ]:
# Вставляем в poems_pos данные по частям речи
insert_pos_into_db(texts)

In [6]:
# Функция выполняющая запрос, выводит запрос, результат запроса и описание запроса.
def query_handler(query, description):
    print(f"\n {description}")
    print(f"Выполняем запрос: {query}")
    mycursor.execute(query)
    result = mycursor.fetchall()
    for row in result:
        print(f'Ответ:{row}')
    return result

In [50]:
# Комбинаторные запросы, запрос с JOIN, вложенный запрос
query_handler(
    "SELECT COUNT(*) FROM poems_metadata WHERE year_end > 1820",
    "1. Общее количество произведений в базе данных, написанных после 1820"
)

query_handler(
    "SELECT title, n_lines FROM poems_metadata WHERE n_lines > 100",
    "2. Произведения с количеством строк больше 100"
)

query_handler(
    "SELECT AVG(n_words) FROM poems_metadata WHERE lifetime_publication = 1",
    "3. Среднее количество слов в произведениях опубликованных при жизни"
)

query_handler(
    "SELECT title, year_start FROM poems_metadata WHERE lifetime_publication = 0",
    "4. Произведения и год их начала написания, опубликованные после смерти автора"
)

query_handler(
    """SELECT year_start, COUNT(*) 
        FROM poems_metadata 
        GROUP BY year_start 
        ORDER BY year_start 
        LIMIT 10""",
    "5. Количество произведений в первые 10 лет от даты начала произведений в датасете"
)

query_handler(
    """SELECT poems_metadata.title, poems_pos.adj_count 
        FROM poems_metadata  
        JOIN poems_pos  ON poems_metadata.uid = poems_pos.uid 
        WHERE poems_pos.most_popular_pos = 'adjective'""",
    "6. Произведения с прилагательными как самой частой частью речи (я люблю красивые описания)"
)

query_handler(
    """SELECT text_content 
        FROM poems_texts
        WHERE uid IN (
        SELECT uid 
        FROM poems_metadata 
        WHERE year_start BETWEEN 1830 AND 1850)  
        LIMIT 10""",
    "7. 10 произведений, написанных между 1830 и 1850")


 1. Общее количество произведений в базе данных, написанных после 1820
Выполняем запрос: SELECT COUNT(*) FROM poems_metadata WHERE year_end > 1820
Ответ:(619,)

 2. Произведения с количеством строк больше 100
Выполняем запрос: SELECT title, n_lines FROM poems_metadata WHERE n_lines > 100
Ответ:('Городок (К***)', 430)
Ответ:('Гусар', 116)
Ответ:('19 октября', 152)
Ответ:('Жених', 184)
Ответ:('<Из Ариостова «Orlando Furioso»>', 111)
Ответ:('К вельможе', 106)
Ответ:('К Жуковскому', 120)
Ответ:('К Наталье', 105)
Ответ:('К Овидию', 104)
Ответ:('К сестре', 122)
Ответ:('Картины', 233)
Ответ:('Кольна (Подражание Оссиану)', 142)
Ответ:('Моему Аристарху', 121)
Ответ:('Моему Аристарху', 116)
Ответ:('«На Испанию родную…»', 115)
Ответ:('Наполеон', 120)
Ответ:('Осень (Отрывок)', 103)
Ответ:('Сестра и братья', 120)
Ответ:('Песнь о вещем Олеге', 102)
Ответ:('Послание Дельвигу', 144)
Ответ:('Послание к Г<алич>у', 145)
Ответ:('Послание к Ю<дину>', 227)
Ответ:('Послание цензору', 126)
Ответ:('Разговор к

[('Вы за «Онегина» советуете, други,\nОпять приняться мне в осенние досуги.\nВы говорите мне: он жив и не женат.\nИтак, еще роман не кончен — это клад:\nВставляй в просторную<?>, вместительную раму\nКартины новые — открой нам диораму:\nПривалит публика, платя тебе за вход —\n(Что даст еще тебе и славу и доход).\n\n[Пожалуй — я бы рад —]\n[Так некогда поэт]',),
 ('<epigraph>Что есть истина?</epigraph>\n<speaker>Друг.</speaker>\nДа, слава в прихотях вольна.\nКак огненный язык, она\nПо избранным главам летает,\nС одной сегодня исчезает\nИ на другой уже видна.\nЗа новизной бежать смиренно\nНарод бессмысленный привык;\nНо нам уж то чело священно,\nНад коим вспыхнул сей язык.\nНа троне, на кровавом поле,\nМеж граждан на чреде иной\nИз сих избранных кто всех боле\nТвоею властвует душой?\n<speaker>Поэт.</speaker>\nВсё он, всё он — пришлец сей бранный,\nПред кем смирилися цари,\nСей ратник, вольностью венчанный,\nИсчезнувший, как тень зари.\n<speaker>Друг.</speaker>\nКогда ж твой ум он поражает

In [12]:
# Смотрим на название одного произведения
query_handler("SELECT title FROM poems_metadata WHERE uid='A39_0'", "Смотрим на название одного произведения по uid")


 Смотрим на название одного произведения по uid
Выполняем запрос: SELECT title FROM poems_metadata WHERE uid='A39_0'
Ответ:('<В альбом>',)


[('<В альбом>',)]

In [13]:
# Мне не понравилось название произведения, там некрасивое форматирование, поменяю его
query_handler("UPDATE poems_metadata SET title = 'В альбом' WHERE uid = 'A39_0'", "Меняем ненравящееся название произведение по uid")


 Меняем ненравящееся название произведение по uid
Выполняем запрос: UPDATE poems_metadata SET title = 'В альбом' WHERE uid = 'A39_0'


[]

In [14]:
# Проверим поменялось ли название
query_handler("SELECT title FROM poems_metadata WHERE uid='A39_0'", "Проверим поменялось ли название произведения по uid")


 Проверим поменялось ли название произведения по uid
Выполняем запрос: SELECT title FROM poems_metadata WHERE uid='A39_0'
Ответ:('В альбом',)


[('В альбом',)]